In [27]:
import os
from keras.preprocessing import image

os.getcwd()
os.getcwd()

path = "./" + "Fuku_merge/Fuku_Apr7Fri_2017_052616_1.tif"
img = image.load_img(path, target_size=(100,100))
print(img)
# img.show()
#print(x[0])
print(type(x))
print(x.shape)
x = image.img_to_array(img)
#print(x[0])
print(type(x))
print(x.shape)

<PIL.Image.Image image mode=RGB size=100x100 at 0x7FE6D29C9F60>
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)


In [28]:
batch_size = 1
num_classes = 2
epochs = 20

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
()

()

In [102]:
import numpy as np
import os
from keras.preprocessing import image

# loadImages
def loadImages(dir:str):
    images = []
    files = []
    for f in os.listdir(dir):
        file = os.path.join(dir, f)
        img = image.load_img(file, target_size=(128,128))
        img_array = image.img_to_array(img)
        images.append(img_array)
        files.append(file)
    return (images, files)


from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 1
num_classes = 2
epochs = 12

# input image dimensions
img_rows, img_cols = 128, 128
# input image channels
img_channels = 3

# 
images_Fuku, files_Fuku = loadImages('Fuku_merge')
images_Kazuma, files_Kazuma = loadImages('Kazuma_merge')

images_Fuku = np.array(images_Fuku)
images_Kazuma = np.array(images_Kazuma)

print(type(images_Fuku))
print(type(images_Kazuma))

print(images_Fuku.shape)
print(images_Kazuma.shape)

print(len(images_Fuku))

# データサイズを合わせる
while(len(images_Fuku) > len(images_Kazuma)):
    i = np.random.randint(len(images_Fuku))
    images_Fuku = np.delete(images_Fuku, i, 0)
    files_Fuku = np.delete(files_Fuku, i, 0)
# print(images_Fuku.shape)
# print(images_Kazuma.shape)

# データを結合
x = np.vstack((images_Fuku, images_Kazuma))
print(x.shape)

# データを標準化
# sklearnのAPIには多次元配列は入力できないので、
# 一旦ベクトル化して処理した後、reshapeして多次元化する
from sklearn import preprocessing
x_tmp = x.reshape(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3])
x_scaled = preprocessing.scale(x_tmp)
x_scaled = x_scaled.reshape(x.shape)
print(x_scaled.shape)

# ラベルデータを作成
num_data = len(images_Fuku)
y = np.hstack((np.zeros(num_data),np.ones(num_data)))

# ラベルデータをカテゴリ表現へ変換
y = keras.utils.to_categorical(y, num_classes)
# print(y)

# ホールドアウト検証のため、訓練データとテストデータに分割
from sklearn.cross_validation import train_test_split
(x_train, x_test, y_train, y_test) = train_test_split(x_scaled, y, test_size = 0.2, random_state=11)
# print(x_train.shape)
# print(x_test.shape)

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

print("x_train.shape[1:]" + str(x_train.shape[1:]))

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                padding='same',
                activation='relu',
                input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(97, 128, 128, 3)
(50, 128, 128, 3)
97
(100, 128, 128, 3)
(100, 128, 128, 3)
x_train shape: (80, 128, 128, 3)
80 train samples
20 test samples
x_train.shape[1:](128, 128, 3)


/home/ujitoko/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/ujitoko/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:177: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 126, 126, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 63, 63, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 254016)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32514176  
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
__________